In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn import datasets
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
import pickle
import joblib

from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.linear_model import SGDRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

In [4]:
diamonds = pd.read_csv('./data/diamonds_full.csv')

In [5]:
diamonds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53940 entries, 0 to 53939
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  53940 non-null  int64  
 1   price       53940 non-null  float64
 2   carat       53940 non-null  float64
 3   cut         53940 non-null  object 
 4   color       53940 non-null  object 
 5   clarity     53940 non-null  object 
 6   depth       53940 non-null  float64
 7   table       53940 non-null  float64
 8   x           53940 non-null  float64
 9   y           53940 non-null  float64
 10  z           53940 non-null  float64
 11  city        53940 non-null  object 
dtypes: float64(7), int64(1), object(4)
memory usage: 4.9+ MB


In [6]:
#diamonds = diamonds.drop(['Unnamed: 0','index_id','cut_id', 'color_id', 'clarity_id'], axis = 1)
diamonds = diamonds.drop(['Unnamed: 0'], axis = 1)

In [7]:
diamonds

,price,carat,cut,color,clarity,depth,table,x,y,z,city
0,4268.0000,1.21,Premium,J,VS2,62.4,58.0,6.83,6.79,4.25,Dubai
1,505.0000,0.32,Very Good,H,VS2,63.0,57.0,4.35,4.38,2.75,Kimberly
2,2686.0000,0.71,Fair,G,VS1,65.5,55.0,5.62,5.53,3.65,Las Vegas
3,738.0000,0.41,Good,D,SI1,63.8,56.0,4.68,4.72,3.00,Kimberly
4,4882.0000,1.02,Ideal,G,SI1,60.5,59.0,6.55,6.51,3.95,Dubai
...,...,...,...,...,...,...,...,...,...,...,...
53935,1662.0140,0.57,Ideal,E,SI1,61.9,56.0,5.35,5.32,3.30,Amsterdam
53936,2536.6023,0.71,Ideal,I,VS2,62.2,55.0,5.71,5.73,3.56,New York City
53937,2886.0555,0.70,Ideal,F,VS1,61.6,55.0,5.75,5.71,3.53,Tel Aviv
53938,2162.7330,0.70,Very Good,F,SI2,58.8,57.0,5.85,5.89,3.45,Surat


In [8]:
diamonds

,price,carat,cut,color,clarity,depth,table,x,y,z,city
0,4268.0000,1.21,Premium,J,VS2,62.4,58.0,6.83,6.79,4.25,Dubai
1,505.0000,0.32,Very Good,H,VS2,63.0,57.0,4.35,4.38,2.75,Kimberly
2,2686.0000,0.71,Fair,G,VS1,65.5,55.0,5.62,5.53,3.65,Las Vegas
3,738.0000,0.41,Good,D,SI1,63.8,56.0,4.68,4.72,3.00,Kimberly
4,4882.0000,1.02,Ideal,G,SI1,60.5,59.0,6.55,6.51,3.95,Dubai
...,...,...,...,...,...,...,...,...,...,...,...
53935,1662.0140,0.57,Ideal,E,SI1,61.9,56.0,5.35,5.32,3.30,Amsterdam
53936,2536.6023,0.71,Ideal,I,VS2,62.2,55.0,5.71,5.73,3.56,New York City
53937,2886.0555,0.70,Ideal,F,VS1,61.6,55.0,5.75,5.71,3.53,Tel Aviv
53938,2162.7330,0.70,Very Good,F,SI2,58.8,57.0,5.85,5.89,3.45,Surat


In [9]:
diamonds_encoded = pd.get_dummies(diamonds, columns=['city', 'cut', 'clarity', 'color'])

In [18]:
diamonds_encoded

,price,carat,depth,table,x,y,z,city_Amsterdam,city_Antwerp,city_Dubai,...,clarity_VS2,clarity_VVS1,clarity_VVS2,color_D,color_E,color_F,color_G,color_H,color_I,color_J
0,4268.0000,1.21,62.4,58.0,6.83,6.79,4.25,False,False,True,...,True,False,False,False,False,False,False,False,False,True
1,505.0000,0.32,63.0,57.0,4.35,4.38,2.75,False,False,False,...,True,False,False,False,False,False,False,True,False,False
2,2686.0000,0.71,65.5,55.0,5.62,5.53,3.65,False,False,False,...,False,False,False,False,False,False,True,False,False,False
3,738.0000,0.41,63.8,56.0,4.68,4.72,3.00,False,False,False,...,False,False,False,True,False,False,False,False,False,False
4,4882.0000,1.02,60.5,59.0,6.55,6.51,3.95,False,False,True,...,False,False,False,False,False,False,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53935,1662.0140,0.57,61.9,56.0,5.35,5.32,3.30,True,False,False,...,False,False,False,False,True,False,False,False,False,False
53936,2536.6023,0.71,62.2,55.0,5.71,5.73,3.56,False,False,False,...,True,False,False,False,False,False,False,False,True,False
53937,2886.0555,0.70,61.6,55.0,5.75,5.71,3.53,False,False,False,...,False,False,False,False,False,True,False,False,False,False
53938,2162.7330,0.70,58.8,57.0,5.85,5.89,3.45,False,False,False,...,False,False,False,False,False,True,False,False,False,False


In [17]:
diamonds_encoded.to_csv('./data/diamonds_full_enc.csv')

In [10]:
X = diamonds_encoded.drop('price', axis=1)
y = diamonds_encoded['price']

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(f"X_train: {X_train.shape}, X_test: {X_test.shape}, y_train: {y_train.shape}, y_test: {y_test.shape}")


X_train: (43152, 39), X_test: (10788, 39), y_train: (43152,), y_test: (10788,)


In [110]:

# Calcular la matriz de correlación
correlation_matrix = diamonds_encoded.corr()

# Seleccionar la variable objetivo (en este ejemplo, 'price') y ordenar las correlaciones en orden descendente
target_correlation = correlation_matrix['price'].sort_values(ascending=False)

# Definir un umbral de correlación por encima del cual consideraremos que las características están altamente correlacionadas con la variable objetivo
correlation_threshold = 0.5

# Filtrar las características que tienen una correlación por encima del umbral
selected_features = target_correlation[abs(target_correlation) > correlation_threshold].index.tolist()

# Eliminar la variable objetivo de la lista de características seleccionadas
selected_features.remove('price')

# Seleccionar solo las características que pasan el umbral de correlación
df_selected = diamonds_encoded[selected_features]


In [111]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Entrenar el modelo de Random Forest
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train_scaled, y_train)

# Realizar predicciones en el conjunto de prueba
y_pred = model.predict(X_test_scaled)

# Calcular el RMSE
rmse = mean_squared_error(y_test, y_pred, squared=False)
print("RMSE:", rmse)

RMSE: 507.4582701114216


/Users/usuario/miniconda3/envs/jupyter/lib/python3.10/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [13]:
diamonds_test = pd.read_csv('./data/diamonds_test.csv')

In [14]:
diamonds_encoded_test = pd.get_dummies(diamonds_test, columns=['city', 'cut', 'clarity', 'color'])


In [16]:
diamonds_encoded_test.to_csv('./data/diamonds_test_enc.csv')

In [114]:
X_prueba = diamonds_encoded_test.drop('id', axis=1)
X_prueba_scaled = scaler.transform(X_prueba)
y_prueba_pred = model.predict(X_prueba_scaled)


In [115]:
solution = []
for i, value in enumerate(y_prueba_pred):
    solution.append((i,value))
    

In [116]:
solution_df = pd.DataFrame(solution)

In [117]:
solution_df.columns = ['id', 'price']

In [118]:
solution_df.to_csv('./data/submission.csv', index=False)

In [119]:
solution_df.tail(30)

,id,price
13455,13455,695.536377
13456,13456,516.904171
13457,13457,1937.963788
13458,13458,1685.355663
13459,13459,3748.563907
13460,13460,2172.292684
13461,13461,1787.699227
13462,13462,2625.716957
13463,13463,809.725114
13464,13464,8402.901046
